In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
print("Cell 2: Installing unsloth and forcing upgrade on all dependencies...")


!pip install "unsloth[colab-new]"

print("Cell 2 Complete: Unsloth and all dependencies force-upgraded.")
print("!!! IMPORTANT: YOU MUST RESTART THE KERNEL NOW BEFORE RUNNING THE NEXT CELL. !!!")

Cell 2: Installing unsloth and forcing upgrade on all dependencies...
Cell 2 Complete: Unsloth and all dependencies force-upgraded.
!!! IMPORTANT: YOU MUST RESTART THE KERNEL NOW BEFORE RUNNING THE NEXT CELL. !!!


In [4]:
# Import the 'os' library to interact with the operating system
import os

# --- Environment Check ---
# We check for an environment variable 'COLAB_GPU'. If it exists, we're in Google Colab.
if 'COLAB_GPU' in os.environ:
    print("Running in Google Colab.")
    # Google Drive mounting happens inside train.py if needed.
    # We set the SAVE_PATH variable here mainly for the post-training steps.
    SAVE_PATH = "/content/drive/MyDrive/llama3_math_checkpoint"

# We check if the /kaggle/input directory exists. If it does, we're in a Kaggle Notebook.
elif os.path.exists('/kaggle/input'):
    print("Running in Kaggle Notebook.")
    # Set the save path to Kaggle's writable directory
    SAVE_PATH = "/kaggle/working/llama3_math_checkpoint"

else:
    print("Running in a local environment.")
    # Set a default path for a local machine
    SAVE_PATH = "llama3_math_checkpoint"

# --- Create Save Directory ---
# 'os.makedirs' creates the directory if it doesn't already exist.
# 'exist_ok=True' prevents an error if the folder is already there.
# We still create the directory here in case post-processing steps need it,
# although train.py will also create it.
os.makedirs(SAVE_PATH, exist_ok=True)

print(f"Post-training model checkpoint location set to: {SAVE_PATH}")
print("Cell 4 Complete: Environment check done.")

Running in Kaggle Notebook.
Post-training model checkpoint location set to: /kaggle/working/llama3_math_checkpoint
Cell 4 Complete: Environment check done.


In [5]:
print("Cell 5: Importing libraries...")
# --- Unsloth ---

from unsloth import FastLanguageModel 

# --- PyTorch ---
import torch  # The core deep learning library

# --- Hugging Face Datasets ---
from datasets import load_dataset 

# --- Hugging Face TRL & Transformers ---

from transformers import AutoTokenizer

# --- Data Handling ---

import pandas as pd  
from tqdm import tqdm  

# --- Garbage Collector for memory management later ---
import gc

print("Cell 5 Complete: Necessary libraries imported.")

Cell 5: Importing libraries...
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-11-01 19:22:27.823091: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762024947.847274     202 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762024947.855282     202 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
Cell 5 Complete: Necessary libraries imported.


In [6]:
# --- Model Configuration ---

# The maximum number of tokens in a sequence.
max_seq_length = 1024

dtype = None

load_in_4bit = True

# --- Load the Model ---
model, tokenizer = FastLanguageModel.from_pretrained(

    model_name = "unsloth/Meta-Llama-3.1-8B",

    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    
)

print("Cell 6 Complete: Base Model and Tokenizer loaded.")

==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Cell 6 Complete: Base Model and Tokenizer loaded.


In [7]:
# --- LoRA Configuration ---
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # Use string "unsloth"
    random_state = 42,
)

print("Cell 7 Complete: LoRA configured (for inspection).")

Unsloth 2025.10.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Cell 7 Complete: LoRA configured (for inspection).


In [8]:
# --- 1. Load the Dataset ---
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# --- 2. Create the Subset ---
shuffled_dataset = dataset.shuffle(seed=42)

train_subset = shuffled_dataset.select(range(50000))
print(f"Using a subset of {len(train_subset)} examples.")



# THE RAW DATA FOR THE FIRST EXAMPLE 
print("\n" + "="*50)
print("--- DEBUG: RAW DATA (EXAMPLE 0) ---")
print("="*50 + "\n")
example = train_subset[0]
print(f"QUESTION:\n{example['question']}\n")
print(f"EXPECTED ANSWER:\n{example['answer']}\n")
print(f"PROVIDED SOLUTION:\n{example['solution']}\n")
print(f"IS_CORRECT (Target Label):\n{example['is_correct']}\n")


# --- 3. Define the Prompt Template ---
training_prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a meticulous math solution verifier. Your task is to carefully analyze the 'Provided Solution' step-by-step to determine if it logically and correctly reaches the 'Expected Answer' for the given 'Question'. Respond ONLY with 'True' if the solution's reasoning AND final result are correct and match the expected answer, otherwise respond ONLY with 'False'.<|eot_id|><|start_header_id|>user<|end_header_id|>
Question:
{}

Expected Answer:
{}

Provided Solution:
{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{}<|eot_id|>"""


# --- 4. Create a Formatting Function ---
def formatting_prompts_func(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    answers = examples["answer"]
    outputs = [str(x) for x in examples["is_correct"]] 

    texts = []
    for question, solution, answer, output in zip(questions, solutions, answers, outputs):
        text = training_prompt_template.format(
            question,
            str(answer),  
            str(solution),
            output        
        )
        texts.append(text)
    return { "text" : texts }


print("\n" + "="*50)
print("--- DEBUG: FINAL FORMATTED PROMPT (EXAMPLE 0) ---")
print("="*50 + "\n")
# Apply the formatting to just the first example
formatted_example_text = formatting_prompts_func(train_subset.select(range(1)))['text'][0]
print(formatted_example_text)
print("\n" + "="*50)
print("--- END OF DEBUG ---")
print("="*50 + "\n")



print("Applying formatting to the rest of the dataset...")
try:
    num_proc = os.cpu_count() // 2 if os.cpu_count() else 4
except:
    num_proc = 4
formatted_dataset = train_subset.map(formatting_prompts_func, batched = True, num_proc=num_proc)

print("✅ Dataset loaded and formatted with prompt template.")

# --- 6. Split into Train and Eval Sets ---
split_dataset = formatted_dataset.train_test_split(test_size=0.1, seed=42)


final_train_dataset = split_dataset["train"]
final_eval_dataset = split_dataset["test"]

print(f"✅ Dataset split into: {len(final_train_dataset)} training examples and {len(final_eval_dataset)} evaluation examples.")
print("Cell 8 Complete: Dataset prepared and split.")



README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Using a subset of 50000 examples.

--- DEBUG: RAW DATA (EXAMPLE 0) ---

QUESTION:
A line is parameterized by
\[\begin{pmatrix} x \\ y \end{pmatrix} = \begin{pmatrix} 2 \\ 3 \end{pmatrix} + t \begin{pmatrix} -1 \\ 5 \end{pmatrix}.\]A second line is parameterized by
\[\begin{pmatrix} x \\ y \end{pmatrix} = \begin{pmatrix} 0 \\ 7 \end{pmatrix} + u \begin{pmatrix} -1 \\ 4 \end{pmatrix}.\]Find the point where the lines intersect.

EXPECTED ANSWER:
(2/3,4/3)

PROVIDED SOLUTION:
First, we need to solve the system of equations
\[
\begin{aligned}
2 - t &= s\\
3 + 5t &= 7 + 4s
\end{aligned}
\]
by eliminating s.
We'll use sympy.
<llm-code>
from sympy import symbols, solve

# define the variables
t, s = symbols('t s')

# define the equations
equation1 = 2 - t - s
equation2 = 3 + 5*t - (7 + 4*s)

# solve the equations
solutions = solve([equation1, equation2], (t, s))
print(solutions)
</llm-code>
<llm-code-output>
{s: 2/3, t: 4/3}
</llm-code-output>
Therefore the point where the lines intersect is \

Map (num_proc=2):   0%|          | 0/50000 [00:00<?, ? examples/s]

✅ Dataset loaded and formatted with prompt template.
✅ Dataset split into: 45000 training examples and 5000 evaluation examples.
Cell 8 Complete: Dataset prepared and split.


In [9]:
%%writefile train.py
# --- 1. IMPORTS ---
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback, AutoTokenizer
import os
from accelerate import Accelerator
import pandas as pd
from tqdm import tqdm


# --- 2. INITIALIZE ACCELERATOR ---

accelerator = Accelerator()
print(f"train.py: Process rank: {accelerator.process_index}, Device: {accelerator.device}, Main process: {accelerator.is_main_process}")


if 'COLAB_GPU' in os.environ:
    print("train.py: Running in Google Colab.")
    print("train.py: Saving to local Colab storage.")
    SAVE_PATH = "llama3_math_checkpoint_colab_local"
elif os.path.exists('/kaggle/input'):
    print("train.py: Running in Kaggle Notebook.")
    SAVE_PATH = "/kaggle/working/llama3_math_checkpoint"
else:
    print("train.py: Running in a local environment.")
    SAVE_PATH = "llama3_math_checkpoint"

if accelerator.is_main_process: os.makedirs(SAVE_PATH, exist_ok=True)
accelerator.wait_for_everyone()
print(f"train.py: Model checkpoint save path: {SAVE_PATH}")


# --- 4. LOAD MODEL AND TOKENIZER ---
max_seq_length = 1024
dtype = None
load_in_4bit = True

# --- Set up model loading arguments ---
load_model_kwargs = {
    "max_seq_length": max_seq_length,
    "dtype": dtype,
    "load_in_4bit": load_in_4bit,
}



if accelerator.num_processes == 1:
    print("Single-GPU setup. Using device_map='auto'.")
    load_model_kwargs["device_map"] = "auto"
else:
    # This is for 2 or more GPUs
    print(f"Multi-GPU setup ({accelerator.num_processes} processes). Using accelerate device_map.")
    load_model_kwargs["device_map"] = {"": accelerator.device}


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    **load_model_kwargs,
    
)

# --- Apply LoRA ---
model = FastLanguageModel.get_peft_model(
    model, r=16, lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0, bias="none", use_gradient_checkpointing="unsloth", random_state=42,
)


# --- 5. PREPARE DATASET ---
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")
shuffled_dataset = dataset.shuffle(seed=42)


train_subset = shuffled_dataset.select(range(50000))


training_prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a meticulous math solution verifier. Your task is to carefully analyze the 'Provided Solution' step-by-step to determine if it logically and correctly reaches the 'Expected Answer' for the given 'Question'. Respond ONLY with 'True' if the solution's reasoning AND final result are correct and match the expected answer, otherwise respond ONLY with 'False'.<|eot_id|><|start_header_id|>user<|end_header_id|>
Question:
{}

Expected Answer:
{}

Provided Solution:
{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{}<|eot_id|>"""


def formatting_prompts_func(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    answers = examples["answer"]
    outputs = [str(x) for x in examples["is_correct"]] 

    texts = []
    for question, solution, answer, output in zip(questions, solutions, answers, outputs):
        text = training_prompt_template.format(
            question,
            str(answer),  
            str(solution),
            output        
        )
        texts.append(text)
    return { "text" : texts }

try: num_proc = os.cpu_count() // 2 if os.cpu_count() else 4
except: num_proc = 4

with accelerator.main_process_first():
    formatted_dataset = train_subset.map(formatting_prompts_func, batched=True, num_proc=num_proc)
    split_dataset = formatted_dataset.train_test_split(test_size=0.1, seed=42) # 10% for eval

final_train_dataset = split_dataset["train"]
final_eval_dataset = split_dataset["test"]
print(f"train.py Process {accelerator.process_index}: Dataset prepared: {len(final_train_dataset)} train / {len(final_eval_dataset)} eval examples.")


# --- 6. SET UP TRAINER ---
training_args = TrainingArguments(
    output_dir="training_outputs",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="adamw_8bit",
    weight_decay=0.01,
    logging_steps=100,
    learning_rate=1e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    seed=42,
    report_to="none",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    ddp_find_unused_parameters=False,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=final_train_dataset,
    eval_dataset=final_eval_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    packing=False, 
)

# --- 7. START TRAINING ---
print(f"--- 🚀 Process {accelerator.process_index}: Starting Model Training ---")
trainer.train()
print(f"--- ✅ Process {accelerator.process_index}: Model Training Complete ---")

# --- 8. SAVE FINAL MODEL ---
accelerator.wait_for_everyone()
if accelerator.is_main_process:
    print(f"Saving final model checkpoint to {SAVE_PATH}")
    trainer.model.save_pretrained(SAVE_PATH)
    tokenizer.save_pretrained(SAVE_PATH)
    print(f"✅ Model adapters and tokenizer saved to: {SAVE_PATH}")
else:
    print(f"Process {accelerator.process_index}: Skipping save.")
accelerator.wait_for_everyone()
print(f"train.py: Process {accelerator.process_index} finished.")



Writing train.py


In [10]:
print("Cell 10: Launching training script...")
import torch
import os


if torch.cuda.is_available():
    num_processes = torch.cuda.device_count()
    print(f"Found {num_processes} GPU(s). Launching with {num_processes} processes.")
    command = f"accelerate launch --num_processes={num_processes} train.py"
else:
    num_processes = 1
    print("No GPU found. Launching with 1 CPU process.")
    command = f"accelerate launch --num_processes={num_processes} --cpu train.py"

print(f"Running command: {command}")


!{command}

print("Cell 10 Complete: Training script finished.")

Cell 10: Launching training script...
Found 2 GPU(s). Launching with 2 processes.
Running command: accelerate launch --num_processes=2 train.py
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be a

In [11]:
import gc
import torch
import os
import glob 


try:
    del model
    del tokenizer
    print("Deleted notebook's model and tokenizer objects.")
except NameError:
    print("Model/Tokenizer already deleted or not defined in this notebook scope.")

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("Cleared notebook memory. Loading trained model from checkpoint for inference...")


kaggle_final_save_path = "/kaggle/working/llama3_math_checkpoint"
colab_final_save_path = "llama3_math_checkpoint_colab_local"

latest_checkpoint_path = "" 


if os.path.exists(kaggle_final_save_path):
    print(f"Found final saved model at (Kaggle): {kaggle_final_save_path}")
    latest_checkpoint_path = kaggle_final_save_path


elif os.path.exists(colab_final_save_path):
    print(f"Found final saved model at (Colab): {colab_final_save_path}")
    latest_checkpoint_path = colab_final_save_path


else:
    print(f"Final save paths not found. Searching for latest epoch checkpoint in 'training_outputs'...")
    
    list_of_checkpoints = glob.glob("training_outputs/checkpoint-*")
    
    if list_of_checkpoints:
        
        latest_checkpoint_path = max(list_of_checkpoints, key=os.path.getctime)
        print(f"Found latest epoch checkpoint at: {latest_checkpoint_path}")
    else:
        print("!!! CRITICAL ERROR: No checkpoints found. Did Cell 10 run correctly? !!!")
        


if latest_checkpoint_path:
    # --- Load the saved model --
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = latest_checkpoint_path, 
        max_seq_length = max_seq_length,      
        dtype = dtype,                        
        load_in_4bit = load_in_4bit,          
    )

    # --- Prepare for Inference ---
    FastLanguageModel.for_inference(model)

    print("✅ Model loaded from checkpoint and prepared for inference.")
    print("Cell 11 Complete: Memory cleared and trained model reloaded.")
else:
    print("--- 🛑 STOPPING: Could not find a model to load. ---")



Deleted notebook's model and tokenizer objects.
Cleared notebook memory. Loading trained model from checkpoint for inference...
Found final saved model at (Kaggle): /kaggle/working/llama3_math_checkpoint
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Model loaded from checkpoint and prepared for inference.
Cell 11 Complete: Memory cleared and trained model reloaded.
